# Classify text
We are going to train a classifier on the tokenized text input, using the [FastText libary](https://fasttext.cc/). 

In addition to the input data file, we give to the command a few hyperparameter values, and we store the binary file representing the learned model as output.

We only learn for a few epochs, to see how the versioning tools work. 


In [ ]:
# Parameters
"""
:param str input_csv_file: Path to input file
:param str out_model_path: Path to model files
:param float learning_rate: Learning rate
:param int epochs: Number of epochs

:dvc-in input_csv_file: ./poc/data/data_train_tokenized.csv
:dvc-out out_model_path: ./poc/data/fasttext_model.bin
:dvc-out: ./poc/data/fasttext_model.vec
:dvc-extra: --learning-rate 0.7 --epochs 20
"""
# Value of parameters for this Jupyter Notebook only
# the notebook is in ./poc/pipeline/notebooks
input_csv_file = "../../data_train_tokenized.csv"
out_model_path = '../../data/fasttext_model'
learning_rate = .7
epochs = 20

In [ ]:
import pandas as pd
import numpy as np
from collections import Counter
from pyfasttext import FastText
import tempfile
import os

In [ ]:
df = pd.read_csv(input_csv_file)

In [ ]:
import json
df['data'] = df['data'].apply(lambda s: json.loads(s.replace("'", '"')))

In [ ]:
with tempfile.TemporaryDirectory() as tmp_dir:
    tmp_path = os.path.join(tmp_dir, 'unigrams')
    with open(tmp_path, 'w') as f:
        for text, _, lab in df.itertuples(index=False, name=None):
            f.write('__label__{} {}\n'.format(lab, ' '.join(text)))
            
    model = FastText()
    # Fastext automatically add .bin at the end of the output model file name
    out_model_path = out_model_path.replace('.bin', '')
    model.supervised(input=tmp_path, output=out_model_path, epoch=epochs, lr=learning_rate)